In [2]:
import json
import yaml
import os
# http://www.utilities-online.info/xmltojson/#.W4B7JZ2xWRs

with open('./aUHF_msgid.json', 'r') as f:
    msgid = json.load(f)

In [18]:

endianess = 'le'
structs = {}

for packet in msgid['Device']['Packet']:
    print(packet['PacType'], packet['PacId'], packet['PacName'])
    
    current_offset=0
    seq = []
    for field in packet['Field']:
        if field['FldByteOrder'] != 'LE':
            print("ERROR: Changed endianess (no LE)!")
            continue

        if (int(field['FldOffset']) != current_offset):
            print("ERROR: Wrong bit offset: ".format(field['FldName'], field['FldOffset'], current_offset))
        
        if field['FldType'] == 'uint':
            attribute = {'type': 'u{:d}'.format(int(field['FldLen'])//8)}
            current_offset += int(field['FldLen'])
        elif field['FldType'] == 'int':
            attribute = {'type': 's{:d}'.format(int(field['FldLen'])//8)}
            current_offset += int(field['FldLen'])
        elif field['FldType'] == 'string':
            attribute = {'type': 'str',
                         'size': int(field['FldLen'])//8,
                         'encoding': 'ASCII'}
            current_offset += int(field['FldLen'])
        elif field['FldType'] == 'bit':
            attribute = {'type': 'b1'}
            current_offset += 1
        elif field['FldType'] == 'float':
            attribute = {'type': 'f{:d}'.format(int(field['FldLen'])//8)}
            current_offset += int(field['FldLen'])
        else:
            print("ERROR: Unsupported field type {}.".format(field['FieldType']))
            continue

        attribute['id'] = field['FldName']
        if 'FldDesc' in field.keys():
            attribute['doc'] = field['FldDesc']
        
        seq.append(attribute)
    
    with open(os.path.join('./structs/', packet['PacName'] + '.ksy'), 'w') as f:
        yaml.dump(seq, f, default_flow_style=False)

    structs[packet['PacName']] = seq

datain 65505 version_sw
datain 16897 UHF_config
datain 16918 beacon
datain 16919 extendedBeacon


In [27]:
seq = structs['beacon']

header = '''meta:
  id: siriussat
  title: Siriussat Telemetry Decoder
  endian: le
seq:'''

print(header)
for f in seq:
    print('- id: {}'.format(f['id'].lower()))
    for k,v in f.items():
        if k == 'id':
            continue
        elif k == 'doc':
            print('  {}: "{}"'.format(k, v))
        else:
            print('  {}: {}'.format(k, v))

meta:
  id: siriussat
  title: Siriussat Telemetry Decoder
  endian: le
seq:
- id: usb1
  type: u2
  doc: "Solar Panel 1 Voltage(mV)"
- id: usb2
  type: u2
  doc: "Solar Panel 2 Voltage(mV)"
- id: usb3
  type: u2
  doc: "Solar Panel 3 Voltage(mV)"
- id: isb1
  type: u2
  doc: "Solar Panel 1 Current(mA)"
- id: isb2
  type: u2
  doc: "Solar Panel 2 Current(mA)"
- id: isb3
  type: u2
  doc: "Solar Panel 3 Current(mA)"
- id: iab
  type: s2
  doc: "Battery Current(mA)"
- id: ich1
  type: u2
  doc: "Channel 1 Current(mA)"
- id: ich2
  type: u2
  doc: "Channel 2 Current(mA)"
- id: ich3
  type: u2
  doc: "Channel 3 Current(mA)"
- id: ich4
  type: u2
  doc: "Channel 4 Current(mA)"
- id: t1_pw
  type: s2
  doc: "Battery temperature 1(deg. С)"
- id: t2_pw
  type: s2
  doc: "Battery temperature 2(deg. С)"
- id: t3_pw
  type: s2
  doc: "Battery temperature 3(deg. С)"
- id: t4_pw
  type: s2
  doc: "Battery temperature 4(deg. С)"
- id: uab_crit
  type: b1
  doc: "Flag "Critical battery voltage""
- id